<a href="https://colab.research.google.com/github/gogimandu1019/bigdata202406/blob/main/type2_3rd(blank).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

컬럼네임이 index : 인덱스가 필요함
y_pred 로 칼럼명을 정확히 맞춰야 함

# 작업형2
- 빅데이터 분석기사 3회 실기
- 제공된 데이터
    - train.csv
    - test.csv
- 예측할 컬럼 : TravelInsurance

### 시험에서 제공된 코드
~~~
import pandas as pd
a = pd.read_csv("train.csv")
b = pd.read_csv('test.csv')

pd.DataFrame(변수).to_csv('000000000.csv')
~~~


### 보험가입 확률을 묻는 문제
~~~
# csv 형태
# index,y_pred
# 0,0.152467
# 1,0.961511
# 2,0.151313
# 3,0.236632
...
# 494,0.121372
# 495,0.965155
# 496,0.986405

~~~


## 데이터 불러오기

In [11]:
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

## EDA

In [12]:
# 크기 확인
train.shape, test.shape #((1490, 10), (497, 9))

((1490, 10), (497, 9))

In [15]:
# 샘플 확인
train.head() #TravelInsurance : 분류
test.head()

,Unnamed: 0,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,920,32,Government Sector,Yes,650000,5,0,No,No
1,777,27,Government Sector,Yes,500000,5,0,No,No
2,696,32,Government Sector,Yes,350000,2,0,No,No
3,907,26,Private Sector/Self Employed,Yes,1400000,6,0,No,Yes
4,675,27,Private Sector/Self Employed,No,350000,3,0,No,No


In [16]:
# 타입 확인
train.info() #Employment Type, GraduateOrNot, FrequentFlyer, EverTravelledAbroad 는 object
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
 9   TravelInsurance      1490 non-null   int64 
dtypes: int64(6), object(4)
memory usage: 116.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           497 non-null    int64 
 1

In [7]:
# 결측치 확인 train
train.isnull().sum()

model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

In [8]:
# 결측치 확인 test
test.isnull().sum()

model           0
year            0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

In [23]:
# 기초 통계 획인
train.describe()
test.describe()

,Unnamed: 0,Age,AnnualIncome,FamilyMembers,ChronicDiseases
count,497.000000,497.000000,4.970000e+02,497.000000,497.000000
mean,995.167002,29.800805,9.378270e+05,4.744467,0.269618
std,595.113355,2.986286,3.788348e+05,1.629211,0.444208
min,5.000000,25.000000,3.000000e+05,2.000000,0.000000
25%,468.000000,28.000000,6.000000e+05,4.000000,0.000000
50%,986.000000,29.000000,9.000000e+05,5.000000,0.000000
75%,1550.000000,33.000000,1.250000e+06,6.000000,1.000000
max,1986.000000,35.000000,1.750000e+06,9.000000,1.000000


In [19]:
# 기초 통계 획인 object
train.describe(include = "O")
test.describe(include = "O") #원핫인코딩에는 문제없을 것으로 보임

,Employment Type,GraduateOrNot,FrequentFlyer,EverTravelledAbroad
count,497,497,497,497
unique,2,2,2,2
top,Private Sector/Self Employed,Yes,No,No
freq,361,422,395,398


In [20]:
# 타겟(레이블)
target = train.pop("TravelInsurance")

## 데이터 전처리 및 피처엔지니어링

In [24]:
# 수치형 데이터와 범주형 데이터 분리
n_train = train.select_dtypes(exclude = "object").copy()
c_train = train.select_dtypes(include = "object").copy()
n_test = test.select_dtypes(exclude = "object").copy()
c_test = test.select_dtypes(include = "object").copy()

In [26]:
# 수치형 변수 스케일링
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
cols = ['Age','AnnualIncome','FamilyMembers','ChronicDiseases']
display(n_train.head())
n_train[cols] = scaler.fit_transform(n_train[cols])
n_test[cols] = scaler.fit_transform(n_test[cols])
n_train.head()

,Unnamed: 0,Age,AnnualIncome,FamilyMembers,ChronicDiseases
0,888,28,1250000,6,1
1,1308,31,1250000,7,1
2,151,29,1200000,7,0
3,396,33,650000,6,1
4,135,28,800000,6,0


,Unnamed: 0,Age,AnnualIncome,FamilyMembers,ChronicDiseases
0,888,-0.25,0.538462,0.5,1.0
1,1308,0.50,0.538462,1.0,1.0
2,151,0.00,0.461538,1.0,0.0
3,396,1.00,-0.384615,0.5,1.0
4,135,-0.25,-0.153846,0.5,0.0


In [27]:
# 범주형 변수 인코딩
c_train = pd.get_dummies(c_train)
c_test = pd.get_dummies(c_test)

In [30]:
# 분리한 데이터 다시 합침
train = pd.concat([n_train, c_train] , axis = 1)
test = pd.concat([n_test, c_test], axis = 1)
print(train.shape, test.shape)
train.head()
train.pop("Unnamed: 0")
test.pop("Unnamed: 0")
print(train.shape, test.shape)

(1490, 13) (497, 13)
(1490, 12) (497, 12)


## 검증데이터 분리

In [31]:
# 검증데이터 분리
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(train, target, test_size = 0.1, random_state = 0)
x_tr.shape, x_val.shape, y_tr.shape, y_val.shape

((1341, 12), (149, 12), (1341,), (149,))

## 모델 학습 및 예측

In [38]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 0)
rf.fit(x_tr, y_tr)
pred = rf.predict_proba(x_val)[:,1]
pred

array([0.7575    , 0.915     , 0.33      , 0.42      , 0.99      ,
       0.207     , 1.        , 0.09166667, 1.        , 1.        ,
       0.728     , 0.03833333, 1.        , 0.39266667, 0.06      ,
       0.44233333, 0.50144048, 0.0475    , 0.40333333, 0.79      ,
       0.204     , 0.145     , 0.176     , 0.0175    , 0.74      ,
       0.015     , 0.10333333, 1.        , 0.12333333, 0.9745    ,
       0.15303571, 1.        , 0.20236905, 0.2722619 , 1.        ,
       0.78216667, 0.22714286, 0.00285714, 0.01      , 0.11616667,
       0.02833333, 0.04583333, 0.23416667, 0.53833333, 0.1725    ,
       0.77      , 0.02      , 0.19      , 0.0575    , 0.28      ,
       0.08619048, 0.00516667, 0.07583333, 0.89216667, 0.2022381 ,
       0.2       , 0.03316667, 0.41      , 0.01275   , 0.01      ,
       0.035     , 0.035     , 0.03      , 1.        , 0.8975    ,
       0.68408442, 0.11416667, 0.05933333, 0.03      , 0.87166667,
       0.02833333, 0.01      , 0.21607648, 0.14514286, 0.01   

In [39]:
# 평가
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, pred) #0.8403954802259886

0.8403954802259886

## 예측

In [41]:
# test 예측
pred = rf.predict_proba(test)[:, 1]
pred

array([0.18      , 0.00847222, 0.03011905, 0.98      , 0.1532619 ,
       1.        , 0.19566667, 0.01      , 0.76333333, 0.        ,
       0.09714286, 0.09660964, 0.29183333, 0.30521429, 0.81      ,
       0.09130952, 0.34      , 0.        , 0.30416667, 0.502     ,
       0.06      , 0.        , 1.        , 0.94      , 0.09660964,
       0.99      , 1.        , 0.0075    , 1.        , 0.97      ,
       0.012     , 0.14964286, 0.022     , 0.48028571, 0.01      ,
       0.88      , 0.17236111, 0.03      , 0.98      , 0.01      ,
       0.0325    , 0.005     , 0.18366667, 0.09660964, 0.97      ,
       0.03316667, 1.        , 0.17      , 0.27397619, 0.00666667,
       0.02      , 0.21207612, 0.46483333, 0.25      , 0.00516667,
       0.0075    , 0.2022381 , 0.93333333, 0.351     , 0.76666667,
       0.54845238, 0.29016667, 0.40858189, 0.35611905, 0.16133333,
       0.075     , 0.46333333, 0.77      , 0.67      , 0.03208333,
       0.62333333, 0.05185714, 0.        , 0.99      , 0.21   

In [43]:
# csv 파일1 생성 (예시와 다른 형태)
submission = pd.DataFrame({'index' : test.index, 'y_pred' : pred}).to_csv("0000.csv", index=False)
submission

In [44]:
# csv 파일2 생성
submission2 = pd.DataFrame({'y_pred' : pred}).reset_index().to_csv("1111.csv", index = False)
submission2

In [45]:
# csv 파일1 확인
pd.read_csv("0000.csv")

,index,y_pred
0,0,0.180000
1,1,0.008472
2,2,0.030119
3,3,0.980000
4,4,0.153262
...,...,...
492,492,0.045333
493,493,0.963333
494,494,0.200000
495,495,0.970000


In [46]:
# csv 파일2 확인
pd.read_csv("1111.csv")

,index,y_pred
0,0,0.180000
1,1,0.008472
2,2,0.030119
3,3,0.980000
4,4,0.153262
...,...,...
492,492,0.045333
493,493,0.963333
494,494,0.200000
495,495,0.970000


## 체점
- 수험자는 알 수 없는 부분임

In [ ]:
# y_test = pd.read_csv("y_test.csv")
# roc_auc_score(y_test, pred)

In [ ]:
# 당신의 score는?: 0.7875693000693